In [ ]:
#default_exp gui

In [ ]:
#hide
from nbdev.showdoc import *

# GUI

In [ ]:
#export
import ipywidgets as widgets

from memery.core import queryFlow
from pathlib import Path
from IPython.display import clear_output



## App design

So what zones do we need for a proper image search app? Two examples come to mind: https://same.energy and https://images.google.com. One is minimalist and brutalist while the other is maximalist in features and refined in design.

Same.energy proves that all you need for image search is a text box, a button, and images. (At least, that's how it started off, and sometimes how it is today. They're A/B testing heavily right now, and we'll see what it evolves into.) If you click on an image result, you are now searching for that image. If you add text, it asks if you want to search for the image with text or just the image. This can lead in any hill-climbing direction the user wants, I suppose. 

Google Images has up to six toolbars overhanging the images, and a complicated lightbox selection window that shows the individual image with a subset of similar images below it. Nested and stacked, providing lots of specific search and filtering capabilities. Not as likely to induce a wikiwalk. They've introduced "collections" now, which are presumably meant to replace the "download to random image folder" functionality of current browsers.

There's also Pinterest, of course, though their engineering is geared more toward gaming Google results than finding the right image by search. Thye have a great browse mode though, and save features. Best of all, they have a goodreads-style user tagging function that allows for a whole different way of sorting images than availableon the other sites.

The functions available from these sites include:

- Text query
- Image query
- Text and image query (totally doable with CLIP vectors)
- Browse visually similar images
- Save images (to cloud mostly)
- Filter images by:
  - Size
  - Color
  - Type
  - Time
  - Usage rights
- Visit homepage for image
- Tagging images
- Searching by tags additively
- Filtering out by tags

Tags and filter categories can both be simulated with CLIP vectors of text tokens like "green" or "noisy" or "illustration" or "menswear". Size of image can be inferred directly from filesize or recorded from bitmap data in the `crafter`. Images as search queries and visually similar image browser are the same function but in different user interaction modes. And image links can be to local files, rather than homepages. Saving images not as relevant in this context, though easily sending them somewhere else is. 

Thus there are really three projects here:
- Basic app functionality with search and grid
- Visually simillar image browsing and search
- Tagging and filtering, auto and manual



## Basic app functionality

We want a unified search bar (variable inputs and a button) and an image grid. And each search should remain accessible after it's run, so we can navigate between and compare. It would be nice to use browser-native navigation but for now, with the plan to run a notebook in Voila and serve locally, better to use `ipywidgets` Tabs mode. Eventually it would also be good to replace or upgrade `ipyplot` or better navigation, but first we should sketch out the new-tab functionality.

Need a tabs output, an event loop, a dictionary of searches run, each search returning a list of filenames to be printed in a sub-output within the tab. All wrapped in a VBox with the inputs.


In [ ]:
filepaths = ['images/Wholesome-Meme-8.jpg', 'images/Wholesome-Meme-1.jpg']

In [ ]:
#export
def get_image(file_loc):
    filepath = Path(file_loc)
    file = open(filepath, 'rb')
    image = widgets.Image(value=file.read(),width=200)
    return(image)

In [ ]:
display(get_image(filepaths[0]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02\x1cICC_PROFILE\…

In [ ]:
imgs = [get_image(f) for f in filepaths]

In [ ]:
#export
def get_grid(filepaths, n=4):
    imgs = [get_image(f) for f in filepaths[:n] if Path(f).exists()]
    grid = widgets.GridBox(imgs, layout=widgets.Layout(grid_template_columns="repeat(auto-fit, 200px)"))
    return(grid)

In [ ]:
get_grid(filepaths)

GridBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x…

In [ ]:
from PIL import Image
from io import BytesIO

In [ ]:
#export
def update_tabs(path, query, n_images, searches, tabs, logbox, image_query=None):
    stem = Path(path.value).stem
    slug = f"{stem}:{str(query.value)}"
    if slug not in searches.keys():
        with logbox:
            print(slug)
            if image_query:
                img = [Image.open(BytesIO(file_info['content'])).convert('RGB') for name, file_info in image_query.items()]
                ranked = queryFlow(path.value, query.value, image_query=img)
            else:
                ranked = queryFlow(path.value, query.value)
            searches[f'{slug}'] = ranked
        
    tabs.children = [get_grid(v, n=n_images.value) for v in searches.values()]
    for i, k in enumerate(searches.keys()):
        tabs.set_title(i, k)
    tabs.selected_index = len(searches)-1

        
#     return(True)

In [ ]:
#export
class appPage():
    
    def __init__(self):
        self.inputs_layout =  widgets.Layout(max_width='80%')

        self.path = widgets.Text(placeholder='path/to/image/folder', value='images/', layout=self.inputs_layout)
        self.query = widgets.Text(placeholder='a funny dog meme', value='a funny dog meme', layout=self.inputs_layout)
        self.image_query = widgets.FileUpload()
        self.n_images = widgets.IntSlider(description='#', value=4, layout=self.inputs_layout)
        self.go = widgets.Button(description="Search", layout=self.inputs_layout)
        self.logbox = widgets.Output(layout=widgets.Layout(max_width='80%', height="3rem", overflow="none"))
        self.all_inputs_layout =  widgets.Layout(max_width='80vw', min_height='40vh', flex_flow='row wrap', align_content='flex-start')

        self.inputs = widgets.Box([self.path, self.query, self.image_query, self.n_images, self.go, self.logbox], layout=self.all_inputs_layout)
        self.tabs = widgets.Tab()
        self.page = widgets.AppLayout(left_sidebar=self.inputs, center=self.tabs)

        self.searches = {}
        self.go.on_click(self.page_update)
        
        display(self.page)

    def page_update(self, b):
        if self.image_query:
            update_tabs(self.path, self.query, self.n_images, self.searches, self.tabs, self.logbox, self.image_query.value)
        else:
            update_tabs(self.path, self.query, self.n_images, self.searches, self.tabs, self.logbox, self.image_query.value)
            
        


In [ ]:
app = appPage()

AppLayout(children=(Box(children=(Text(value='images/', layout=Layout(max_width='80%'), placeholder='path/to/i…